<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Download-the-data-from-NCBI" data-toc-modified-id="Download-the-data-from-NCBI-0.1">Download the data from NCBI</a></span><ul class="toc-item"><li><span><a href="#Compression" data-toc-modified-id="Compression-0.1.1">Compression</a></span><ul class="toc-item"><li><span><a href="#Cleanup" data-toc-modified-id="Cleanup-0.1.1.1">Cleanup</a></span></li></ul></li></ul></li></ul></li><li><span><a href="#References" data-toc-modified-id="References-1">References</a></span></li></ul></div>

## Download the data from NCBI

We will use data from <a name="ref-1"/>[(Stadhouders R, Vidal E, Serra F, Di Stefano B et al. 2018)](#cite-ralph), which comes from mouse cells where Hi-C experiment where conducted in different states during highly-efficient somatic cell reprogramming.

The data can be downloaded from:

https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE53463

Once downloaded the files can be passed to the FASTQ format in order for TADbit to read them.

The easiest way to download the data might be through the `fastq-dump` program from the SRA Toolkit (http://www.ncbi.nlm.nih.gov/Traces/sra/sra.cgi?cmd=show&f=software&m=software&s=software) that we installed with `conda` in notebook A1.

We download 100M reads for each of 4 replicates (2 replicates from B cells and 2 from Pluripotent Stem Cells),and organize each in two files, one per read-end (this step is long and can take __up to 6 hours__):

In [1]:
%%bash

mkdir -p FASTQs

fastq-dump -A SRR5344921 -DQ '+' --defline-seq '@$ac.$si' -X 100000000 --split-files --outdir FASTQs/
mv FASTQs/SRR5344921_1.fastq FASTQs/mouse_B_rep1_1.fastq
mv FASTQs/SRR5344921_2.fastq FASTQs/mouse_B_rep1_2.fastq

fastq-dump -A SRR5344925 -DQ '+' --defline-seq '@$ac.$si' -X 100000000 --split-files --outdir FASTQs/
mv FASTQs/SRR5344925_1.fastq FASTQs/mouse_B_rep2_1.fastq
mv FASTQs/SRR5344925_2.fastq FASTQs/mouse_B_rep2_2.fastq

fastq-dump -A SRR5344969 -DQ '+' --defline-seq '@$ac.$si' -X 100000000 --split-files --outdir FASTQs
mv FASTQs/SRR5344969_1.fastq FASTQs/mouse_PSC_rep1_1.fastq
mv FASTQs/SRR5344969_2.fastq FASTQs/mouse_PSC_rep1_2.fastq

fastq-dump -A SRR5344973 -DQ '+' --defline-seq '@$ac.$si' -X 100000000 --split-files --outdir FASTQs/
mv FASTQs/SRR5344973_1.fastq FASTQs/mouse_PSC_rep2_1.fastq
mv FASTQs/SRR5344973_2.fastq FASTQs/mouse_PSC_rep2_2.fastq

Read 100000000 spots for SRR5344921
Written 100000000 spots for SRR5344921
Read 100000000 spots for SRR5344925
Written 100000000 spots for SRR5344925
Read 100000000 spots for SRR5344969
Written 100000000 spots for SRR5344969
Read 100000000 spots for SRR5344973
Written 100000000 spots for SRR5344973


2018-09-28T09:50:43 fastq-dump.2.9.1 warn: block-size in local file 131072 does not match requested value 32768
2018-09-28T09:50:44 fastq-dump.2.9.1 warn: block-size in local file 131072 does not match requested value 32768
2018-09-28T10:15:14 fastq-dump.2.9.1 warn: block-size in local file 131072 does not match requested value 32768
2018-09-28T10:15:14 fastq-dump.2.9.1 warn: block-size in local file 131072 does not match requested value 32768
2018-09-28T10:39:42 fastq-dump.2.9.1 warn: block-size in local file 131072 does not match requested value 32768
2018-09-28T10:39:43 fastq-dump.2.9.1 warn: block-size in local file 131072 does not match requested value 32768
2018-09-28T11:02:57 fastq-dump.2.9.1 warn: block-size in local file 131072 does not match requested value 32768
2018-09-28T11:02:58 fastq-dump.2.9.1 warn: block-size in local file 131072 does not match requested value 32768


Files are renamed for convenience.

_Note: alternatively you can also directly download the FASTQ from http://www.ebi.ac.uk/_

### Compression

Each of these 8 files, contains 100M reads of 75 nucleotides each, and occupies ~17 Gb (total 130 Gb).

Internally we use DSRC <a name="ref-4"/>[(Roguski and Deorowicz, 2014)](#cite-roguski2014dsrc) that allows better compression ration and, more importantly, faster decompression:

In [2]:
%%bash

dsrc c -t8 FASTQs/mouse_B_rep1_1.fastq FASTQs/mouse_B_rep1_1.fastq.dsrc
dsrc c -t8 FASTQs/mouse_B_rep1_2.fastq FASTQs/mouse_B_rep1_2.fastq.dsrc
dsrc c -t8 FASTQs/mouse_B_rep2_1.fastq FASTQs/mouse_B_rep2_1.fastq.dsrc
dsrc c -t8 FASTQs/mouse_B_rep2_2.fastq FASTQs/mouse_B_rep2_2.fastq.dsrc
dsrc c -t8 FASTQs/mouse_PSC_rep1_1.fastq FASTQs/mouse_PSC_rep1_1.fastq.dsrc
dsrc c -t8 FASTQs/mouse_PSC_rep1_2.fastq FASTQs/mouse_PSC_rep1_2.fastq.dsrc
dsrc c -t8 FASTQs/mouse_PSC_rep2_1.fastq FASTQs/mouse_PSC_rep2_1.fastq.dsrc
dsrc c -t8 FASTQs/mouse_PSC_rep2_2.fastq FASTQs/mouse_PSC_rep2_2.fastq.dsrc

After compression we reduce the total size to 27 Gb (__20% of the original size, and dsrc ensures fast reading of the compressed data__)

_Note:_
   - _using gzip  instead reduces size to ~38 Gb (occupies ~40% more than dsrc compressed files)_
   - _using bzip2 instead reduces size to ~31 Gb (occupies ~15% more than dsrc compressed files)_

  _Both are much slower to generate and read_

#### Cleanup

In [3]:
%%bash

rm -f FASTQs/mouse_B_rep1_1.fastq
rm -f FASTQs/mouse_B_rep1_2.fastq
rm -f FASTQs/mouse_B_rep2_1.fastq
rm -f FASTQs/mouse_B_rep2_2.fastq
rm -f FASTQs/mouse_PSC_rep1_1.fastq
rm -f FASTQs/mouse_PSC_rep1_2.fastq
rm -f FASTQs/mouse_PSC_rep2_1.fastq
rm -f FASTQs/mouse_PSC_rep2_2.fastq

# References

<a name="cite-ralph"/><sup>[^](#ref-1) </sup>Stadhouders R, Vidal E, Serra F, Di Stefano B et al. 2018. _Transcription factors orchestrate dynamic interplay between genome topology and gene regulation during cell reprogramming_.

<a name="cite-roguski2014dsrc"/><sup>[^](#ref-4) </sup>Roguski, \Lukasz and Deorowicz, Sebastian. 2014. _DSRC 2—Industry-oriented compression of FASTQ files_.



<!--bibtex
@article{le2014distinct,
  title={Distinct structural transitions of chromatin topological domains correlate with coordinated hormone-induced gene regulation},
  author={Le Dily, Fran{\c{c}}ois and Ba{\`u}, Davide and Pohl, Andy and Vicent, Guillermo P and Serra, Fran{\c{c}}ois and Soronellas, Daniel and Castellano, Giancarlo and Wright, Roni HG and Ballare, Cecilia and Filion, Guillaume and others},
  journal={Genes \& development},
  volume={28},
  number={19},
  pages={2151--2162},
  year={2014},
  publisher={Cold Spring Harbor Lab}
}
@article{roguski2014dsrc,
  title={DSRC 2—Industry-oriented compression of FASTQ files},
  author={Roguski, {\L}ukasz and Deorowicz, Sebastian},
  journal={Bioinformatics},
  volume={30},
  number={15},
  pages={2213--2215},
  year={2014},
  publisher={Oxford Univ Press}
}
-->